===== TURN ON POWERTOY AWAKE to KEEP CONNECTION ALIVE =====


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
!pip install cloudscraper

In [ ]:
 !pip install curl_cffi pandas lxml

In [ ]:
from curl_cffi import requests
import pandas as pd
from io import StringIO


# def scrape_finviz_curl():
#     url = (
#         "https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap"
#         "&c=0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,14,130,131,"
#         "15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,"
#         "26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,"
#         "45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,"
#         "70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66,103,100,"
#         "107,108,109,112,113,114,115,116,117,120,121,122,105&r=921"
#     )

#     try:
#         print("Fetching data with curl_cffi (mimicking Chrome)...")
#         # 'impersonate' makes the server think we are a real Chrome browser
#         response = requests.get(url, impersonate="chrome120")

#         if response.status_code != 200:
#             print(f"Failed. Status Code: {response.status_code}")
#             return None

#         # Parse HTML
#         dfs = pd.read_html(StringIO(response.text))

#         # Find the right table (usually the largest one)
#         screener_df = max(dfs, key=len)

#         # Cleanup headers if needed
#         if "Ticker" not in screener_df.columns:
#             screener_df.columns = screener_df.iloc[0]
#             screener_df = screener_df[1:]

#         screener_df = screener_df.reset_index(drop=True)

#         print(f"Success! Scraped {len(screener_df)} rows.")
#         print(screener_df.head())

#         return screener_df

#     except Exception as e:
#         print(f"Error: {e}")
#         # Debug tip: If this fails, Finviz might be serving a CAPTCHA page.


# if __name__ == "__main__":
#     scrape_finviz_curl()

In [ ]:
from curl_cffi import requests
import pandas as pd
from io import StringIO


def scrape_finviz_curl(url):
    # url = (
    #     "https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap"
    #     "&c=0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,14,130,131,"
    #     "15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,"
    #     "26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,"
    #     "45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,"
    #     "70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66,103,100,"
    #     "107,108,109,112,113,114,115,116,117,120,121,122,105&r=921"
    # )

    try:
        print("Fetching data with curl_cffi (mimicking Chrome)...")
        # 'impersonate' makes the server think we are a real Chrome browser
        response = requests.get(url, impersonate="chrome120")

        if response.status_code != 200:
            print(f"Failed. Status Code: {response.status_code}")
            return None

        # Parse HTML
        dfs = pd.read_html(StringIO(response.text))

        # Find the right table (usually the largest one)
        screener_df = max(dfs, key=len)

        # Cleanup headers if needed
        if "Ticker" not in screener_df.columns:
            screener_df.columns = screener_df.iloc[0]
            screener_df = screener_df[1:]

        screener_df = screener_df.reset_index(drop=True)

        print(f"Success! Scraped {len(screener_df)} rows.")
        print(screener_df.head())

        return screener_df

    except Exception as e:
        print(f"Error: {e}")
        # Debug tip: If this fails, Finviz might be serving a CAPTCHA page.


# if __name__ == "__main__":
#     scrape_finviz_curl()

==== SET Download Location ====

In [ ]:
download_to_PC = True
download_to_Google_Drive = False

# download_to_PC = False
# download_to_Google_Drive = True

===== SET download_to_PC and end_index =====

In [ ]:
# symbols: The string (or sequence) to slice.
# start_index: Optional. The starting index of the slice. If None, defaults to 0.
# end_index: Optional. The ending index of the slice (exclusive). If None, defaults to the end of the string.
# step_value: Optional. The step value for the slice. If None, defaults to 1.

# True download to PC, False download to Google Drive
download_to_PC = True  # True download to PC
# download_to_PC = False  # True download to Google Drive

# Download all if end_iddex = None
end_index = None
# end_index = 3

In [ ]:
start_index = None

step_value = None
slice_obj = slice(start_index, end_index, step_value)  # Create a slice object

print(f"slice of symbols: symbols[{slice_obj}]")

In [ ]:
! pip install fake-useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent


# selector = '.styled-table-new'

# selector = 'screener-table'

# screener-table
# screener-table


ua = UserAgent()  # Initialize UserAgent


# def download_table(url, selector):
def scrape_finviz_curl(url, selector):
    """
    Downloads table data from a Yahoo Finance page with rate limiting.
    """
    try:
        # Add a User-Agent header to mimic a browser
        # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}  # Example User-Agent
        new_user_agent = ua.random
        headers = {"User-Agent": new_user_agent}
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# selector = '.styled-table-new'

selector = "screener-table"


ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Yahoo Finance page with rate limiting.
    """
    try:
        # Add a User-Agent header to mimic a browser
        # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}  # Example User-Agent
        new_user_agent = ua.random
        headers = {"User-Agent": new_user_agent}
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
# all_stocks_and_etfs_columns
# https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap&r=721&c=0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,75,14,130,131,15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66,103,100,107,108,109,112,113,114,115,116,117,120,121,122,105

In [ ]:
# all_etf_columns_sorted_by_AUM
# https://finviz.com/screener.ashx?v=152&ft=4&o=-e.assetsundermanagement&c=0,1,2,3,4,5,129,75,14,130,131,31,84,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,70,80,83,60,61,63,64,67,81,86,87,88,65,66,103,100,107,108,109,112,113,114,115,116,117,120,121,122,105

In [ ]:
# all_stock_columns_sorted_by_market_cap
# https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap&c=0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,75,14,130,131,15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66

In [ ]:
# url_all_columns = '0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,75,14,130,131,15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66,103,100,107,108,109,112,113,114,115,116,117,120,121,122,105'

In [ ]:
# remove dupicate Dividend column 75 with $ values, kept Dividend column with % value
url_all_columns = "0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,14,130,131,15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66,103,100,107,108,109,112,113,114,115,116,117,120,121,122,105"

In [ ]:
print(f"download the top 1160 stocks by market cap")
url_mktcap = "https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap&c="
# url_mktcap_all_columns = '0,1,2,79,3,4,5,129,6,7,8,9,10,11,12,13,73,74,75,14,130,131,15,16,77,17,18,19,20,21,23,22,132,133,82,78,127,128,24,25,85,26,27,28,29,30,31,84,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,68,70,80,83,76,60,61,62,63,64,67,69,81,86,87,88,65,66'
url_mktcap_rows = [
    "&r=1",
    "&r=21",
    "&r=41",
    "&r=61",
    "&r=81",
    "&r=101",
    "&r=121",
    "&r=141",
    "&r=161",
    "&r=181",
    "&r=201",
    "&r=221",
    "&r=241",
    "&r=261",
    "&r=281",
    "&r=301",
    "&r=321",
    "&r=341",
    "&r=361",
    "&r=381",
    "&r=401",
    "&r=421",
    "&r=441",
    "&r=461",
    "&r=481",
    "&r=501",
    "&r=521",
    "&r=541",
    "&r=561",
    "&r=581",
    "&r=601",
    "&r=621",
    "&r=641",
    "&r=661",
    "&r=681",
    "&r=701",
    "&r=721",
    "&r=741",
    "&r=761",
    "&r=781",
    "&r=801",
    "&r=821",
    "&r=841",
    "&r=861",
    "&r=881",
    "&r=901",
    "&r=921",
    "&r=941",
    "&r=961",
    "&r=981",
    "&r=1001",
    "&r=1021",
    "&r=1041",
    "&r=1061",
    "&r=1081",
    "&r=1101",
    "&r=1121",
    "&r=1141",
]

In [ ]:
print(f"download the top 400 ETFs by AUM")
url_etfs = "https://finviz.com/screener.ashx?v=152&ft=4&o=-e.assetsundermanagement&c="
# url_etfs_all_columns = '0,1,2,3,4,5,129,75,14,130,131,31,84,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,134,125,126,59,70,80,83,60,61,63,64,67,81,86,87,88,65,66,103,100,107,108,109,112,113,114,115,116,117,120,121,122,105'
url_etfs_rows = [
    "&r=1",
    "&r=21",
    "&r=41",
    "&r=61",
    "&r=81",
    "&r=101",
    "&r=121",
    "&r=141",
    "&r=161",
    "&r=181",
    "&r=201",
    "&r=221",
    "&r=241",
    "&r=261",
    "&r=281",
    "&r=301",
    "&r=321",
    "&r=341",
    "&r=361",
    "&r=381",
]

In [ ]:
urls_mktcap = []

for _rows in url_mktcap_rows:
    # _url = url_mktcap + url_mktcap_all_columns + _rows
    _url = url_mktcap + url_all_columns + _rows
    urls_mktcap.append(_url)

print(f"Total number of urls: {len(urls_mktcap)}")
print(f"First 3 urls:\n{urls_mktcap[0:3]}")  # Print the length of the list of url)

In [ ]:
urls_etfs = []

for _rows in url_etfs_rows:
    # _url = url_etfs + url_etfs_all_columns + _rows
    _url = url_etfs + url_all_columns + _rows
    urls_etfs.append(_url)

print(f"Total number of urls: {len(urls_etfs)}")
print(f"First 3 urls:\n{urls_etfs[0:3]}")  # Print the length of the list of url)

In [ ]:
import random

print(
    f"===== -marketcap and -e.assetsundermanagement sort marketcap and AUM in descendiing order ====="
)
print(f"===== download 1160 largest marketcap stocks and 400 largest AUM ETFs =====")
urls = urls_mktcap + urls_etfs
shuffled_urls = random.sample(urls, len(urls))
print(f"len(shuffled_urls): {len(shuffled_urls)}")
print(f"shuffled_urls[0:10]")
for url in shuffled_urls[0:10]:
    print(f"{url}")

In [ ]:
# test_urls = [
#   'https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap&c=0,1,2,3,4,6,14,42,43,44,45,46,47,48,49,50,51,52,53,54,57,58,59,67,65,66,103,100,109,120,121,122&r=1',
#   'https://finviz.com/screener.ashx?v=152&ft=4&o=-e.assetsundermanagement&c=0,1,2,3,4,6,14,42,43,44,45,46,47,48,49,50,51,52,53,54,57,58,59,67,65,66,103,100,109,120,121,122&r=101',
#   'https://finviz.com/screener.ashx?v=152&ft=4&o=-marketcap&c=0,1,2,3,4,6,14,42,43,44,45,46,47,48,49,50,51,52,53,54,57,58,59,67,65,66,103,100,109,120,121,122&r=601',
#   ]

In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# for url in test_urls:
# for url in shuffled_urls[0:3]:
for url in shuffled_urls[slice_obj]:
    # Introduce a delay between requests (adjust as needed)
    delay_seconds = random.uniform(2, 4.5)  # Sleep between 2 and 5 seconds
    # print(f"Downloading {symbol}. Sleeping for {delay_seconds:.2f} seconds...")
    processed_count += 1
    print(
        f"Downloading {url}. Sleeping for {delay_seconds:.2f} seconds.  Processed {processed_count} / {total_urls_to_download} urls"
    )
    time.sleep(delay_seconds)

    # selector = '#screener-output'
    # selector = "table.styled-table-new.screener_table"
    selector = "table.styled-table-new.is-rounded.is-tabular-nums.w-full.screener_table"
    # selector =
    df_temp = scrape_finviz_curl(url)
    # df_temp = download_table(url, selector)

    if df_temp is not None:
        df = pd.concat([df, df_temp], ignore_index=True)
    else:
        print(f"Failed to download data for {url}")

In [ ]:
import requests
from fake_useragent import UserAgent

ua = UserAgent()
url_to_inspect = shuffled_urls[0]

print(f"Re-inspecting URL: {url_to_inspect}")

try:
    headers = {"User-Agent": ua.random}
    response = requests.get(url_to_inspect, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    print(f"Length of response content: {len(response.content)} bytes")
    print("First 500 characters of response content:")
    print(response.text[:500])

except requests.exceptions.RequestException as e:
    print(f"Error during request: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

ua = UserAgent()
url_to_inspect = shuffled_urls[0] # Use the first URL from the shuffled list

print(f"Inspecting URL: {url_to_inspect}")

try:
    headers = {"User-Agent": ua.random}
    response = requests.get(url_to_inspect, headers=headers)
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Try to find the table by common Finviz attributes
    # The main screener table usually has these attributes
    table_body = soup.find('table', attrs={'width': '100%', 'cellpadding': '3', 'cellspacing': '1'})

    if table_body:
        print("Found a potential table using attributes width='100%', cellpadding='3', cellspacing='1'.")
        # You can print a snippet of the table or its rows to confirm
        # print(table_body.prettify())
        # To get the column headers as a hint, let's try to extract them
        headers = [th.text.strip() for th in table_body.find_all('th')]
        print(f"Table Headers: {headers}")
        print("\nBased on this, the correct selector appears to be 'table[width="100%"][cellpadding="3"][cellspacing="1"]'.")
        print("I will update the `selector` variable in cell `HTCD6Lo1wHF4`.")

    else:
        print("Could not find table using common Finviz attributes. Further manual inspection of HTML may be needed.")
        print("Printing first 5000 characters of the page source for manual inspection:")
        print(soup.prettify()[:5000]) # Print a snippet of the HTML

except requests.exceptions.RequestException as e:
    print(f"Error during request: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import os

ua = UserAgent()
url_to_inspect = shuffled_urls[0]  # Use the first URL from the shuffled list

print(f"Inspecting URL: {url_to_inspect}")

try:
    headers = {"User-Agent": ua.random}
    response = requests.get(url_to_inspect, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    soup = BeautifulSoup(response.content, "html.parser")

    # Try to find the table by common Finviz attributes
    # The main screener table usually has these attributes
    table_body = soup.find(
        "table", attrs={"width": "100%", "cellpadding": "3", "cellspacing": "1"}
    )

    if table_body:
        print(
            "Found a potential table using attributes width='100%', cellpadding='3', cellspacing='1'."
        )
        headers = [th.text.strip() for th in table_body.find_all("th")]
        print(f"Table Headers: {headers}")
        print(
            f'\nBased on this, the correct selector appears to be \'table[width="100%"][cellpadding="3"][cellspacing="1"]\'.'
        )
        print("I will update the `selector` variable in cell `HTCD6Lo1wHF4`.")

    else:
        print(
            "Could not find table using common Finviz attributes. Further manual inspection of HTML may be needed."
        )
        html_output_filename = "finviz_page_source.html"
        with open(html_output_filename, "w", encoding="utf-8") as f:
            f.write(soup.prettify())
        print(
            f"Full page source saved to {html_output_filename} for manual inspection."
        )
        print("Please review this file to find the correct CSS selector for the table.")

except requests.exceptions.RequestException as e:
    print(f"Error during request: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
import datetime
import pytz

pst = pytz.timezone("America/Los_Angeles")  # or 'US/Pacific'
current_date_pst = datetime.datetime.now(pst).strftime("%Y-%m-%d")

# Save the DataFrame as a Parquet file
df_finviz_filename = f"df_finviz_{current_date_pst}_stocks_etfs.parquet"

# Save the ticker as a text file
ticker_filename = f"ticker_{current_date_pst}_stocks_etfs.csv"

print(f"df_finviz_filename: {df_finviz_filename}")
print(f"ticker_filename: {ticker_filename}")

In [ ]:
import pyarrow

df.to_parquet(df_finviz_filename, engine="pyarrow", compression="zstd")

In [ ]:
# prompt: save ticker in df["Ticker'] as a csv file

# Assuming df is your DataFrame and it contains a column named "Ticker"
df.to_csv(ticker_filename, columns=["Ticker"], header=False, index=False)

In [ ]:
pd.set_option("display.max_columns", None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option("display.width", 2000)

print(df.head())

In [ ]:
def zip_files(filenames, zip_filename):
    """Zips multiple files into a single archive."""
    import zipfile

    with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
        for filename in filenames:
            zipf.write(filename)
    print(f"Files zipped into {zip_filename}")

Download to PC

In [ ]:
if download_to_PC:
    from google.colab import files

    # Example usage, replace with actual filenames
    filenames = [df_finviz_filename, ticker_filename]
    zip_filename = (
        f"{current_date_pst}_combined_files.zip"  # Name for the zipped archive
    )

    zip_files(filenames, zip_filename)

    # Download the zip file
    files.download(zip_filename)

Downlaod to Google Drive

In [ ]:
if download_to_Google_Drive:
    import shutil

    # Define the source and destination paths
    source_path = zip_filename
    destination_path = "/content/drive/MyDrive/00_colab_download/" + zip_filename

    # Copy the file
    shutil.copy(source_path, destination_path)

    print(f"File '{zip_filename}' copied to Google Drive.")

In [ ]:
# ======================

# Task
The previous attempt to scrape Finviz data failed because the HTML table could not be found with the `.styled-table-new` CSS selector. An inspection of the page source suggested that the correct selector should be `'table[width="100%"][cellpadding="3"][cellspacing="1"]'`.

To resolve this, I will:

1.  Update the `selector` variable in cell `HTCD6Lo1wHF4` to the newly identified selector.
2.  Modify the `download_table` function in cell `HTCD6Lo1wHF4` to include a `Referer` header and print the HTTP status code and a snippet of the response content for improved debugging.
3.  Execute the modified `download_table` function on a single URL (`shuffled_urls[0]`) to verify that it can now successfully retrieve the table.
4.  If the test is successful, re-run the main data scraping loop in cell `sp4KYbgEcotc` to download all the data.
5.  Summarize the resolution of the scraping issue and confirm the successful download of Finviz data.

## Update Download Function

### Subtask:
Modify the `download_table` function in cell `HTCD6Lo1wHF4` to include a more complete set of request headers (including a specific User-Agent and Referer) and print the HTTP status code and initial response content for debugging.


**Reasoning**:
I need to update the `selector` variable and modify the `download_table` function in cell `HTCD6Lo1wHF4` to include a `Referer` header and print debugging information (status code and initial response content).



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Yahoo Finance page with rate limiting.
    """
    try:
        # Add a User-Agent header to mimic a browser
        new_user_agent = ua.random
        headers = {
            "User-Agent": new_user_agent,
            "Referer": "https://finviz.com/screener.ashx",
        }
        response = requests.get(url, headers=headers)
        print(f"HTTP Status Code: {response.status_code}")
        print(f"Initial response content (first 500 chars): {response.text[:500]}")
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

**Reasoning**:
The previous code block successfully updated the `selector` variable and modified the `download_table` function as requested. Now, I will proceed to the next instruction in the main task, which is to execute the modified `download_table` function on a single URL to verify its functionality.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The previous test download showed an empty 'Initial response content' despite an HTTP Status Code of 200, and the table selector still failed to find the table body. This suggests the content being received by `requests.get` might be empty or problematic. I will modify the `download_table` function to explicitly print the length of the raw response content to diagnose if the content itself is empty.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Yahoo Finance page with rate limiting.
    """
    try:
        # Add a User-Agent header to mimic a browser
        new_user_agent = ua.random
        headers = {
            "User-Agent": new_user_agent,
            "Referer": "https://finviz.com/screener.ashx",
        }
        response = requests.get(url, headers=headers)
        print(f"HTTP Status Code: {response.status_code}")
        print(
            f"Length of response content: {len(response.content)} bytes"
        )  # Added for debugging
        print(f"Initial response content (first 500 chars): {response.text[:500]}")
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

**Reasoning**:
The previous code block was intended to update the `download_table` function with additional debugging information, including the length of the response content. To ensure this updated function is active in the environment, I am providing the same code block again.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Yahoo Finance page with rate limiting.
    """
    try:
        # Add a User-Agent header to mimic a browser
        new_user_agent = ua.random
        headers = {
            "User-Agent": new_user_agent,
            "Referer": "https://finviz.com/screener.ashx",
        }
        response = requests.get(url, headers=headers)
        print(f"HTTP Status Code: {response.status_code}")
        print(
            f"Length of response content: {len(response.content)} bytes"
        )  # Added for debugging
        print(f"Initial response content (first 500 chars): {response.text[:500]}")
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

**Reasoning**:
Now that the `download_table` function has been updated to include printing the length of the response content for debugging, I will re-run the test with a single URL to observe the new debugging output.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The previous test indicated that `requests.get` is returning a `200` status code but with an empty response body (`Length of response content: 0 bytes`). This suggests Finviz might be blocking or serving an empty page to automated requests. I will add more browser-like headers (Accept-Language, Accept-Encoding) to the `download_table` function to make the request appear more legitimate and explicitly handle cases where no content is received before attempting to parse with BeautifulSoup.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Yahoo Finance page with rate limiting.
    """
    try:
        # Add a User-Agent header to mimic a browser
        new_user_agent = ua.random
        headers = {
            "User-Agent": new_user_agent,
            "Referer": "https://finviz.com/screener.ashx",
            "Accept-Language": "en-US,en;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
        }
        response = requests.get(url, headers=headers, timeout=10)  # Added a timeout
        print(f"HTTP Status Code: {response.status_code}")
        print(
            f"Length of response content: {len(response.content)} bytes"
        )  # Added for debugging
        print(f"Initial response content (first 500 chars): {response.text[:500]}")

        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        if len(response.content) == 0:
            print("Error: Received empty response content from the server.")
            return None

        soup = BeautifulSoup(response.content, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

**Reasoning**:
The previous code block updated the `download_table` function to include more browser-like headers and a timeout. To verify the changes and proceed with testing, I will re-run the single URL test.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

## Implement Headless Browser Scraping

### Subtask:
Modify the `download_table` function to use Selenium with a headless Chrome browser to bypass Finviz's anti-scraping measures and successfully retrieve page content.


**Reasoning**:
The first step is to install the `selenium` and `webdriver-manager` libraries as they are necessary for headless browser scraping with Chrome.



In [ ]:
pip install selenium webdriver-manager

**Reasoning**:
The previous attempts with `requests` failed to retrieve the page content, indicating that Finviz is likely blocking standard HTTP requests or requires JavaScript rendering. Therefore, I will implement headless browser scraping using Selenium to bypass these restrictions. This step involves importing the necessary Selenium modules and updating the `download_table` function as specified in the instructions to use a headless Chrome WebDriver.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")

        # Initialize WebDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        # The selector here targets the table element itself, not its content
        wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except Exception as e:
        print(f"An error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
Now that the `download_table` function has been updated to use Selenium, I will re-run the test with a single URL to verify that it can successfully retrieve table data from Finviz.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The previous attempt to use Selenium failed because the Chrome binary was not found. To resolve this, I need to install Google Chrome in the Colab environment. After installation, the `download_table` function can properly utilize the Chrome driver.



In [ ]:
import sys

# Install Chrome in the Colab environment
!apt-get update
!apt-get install -y google-chrome-stable

print("Google Chrome installed successfully.")


**Reasoning**:
The previous attempt to install `google-chrome-stable` failed because the package could not be located. I will try an alternative installation method by downloading the `.deb` package directly and installing it with `dpkg`. I will also add the Chrome binary path to the system path, although `webdriver-manager` should ideally find it.



In [ ]:
import sys
import os

# Install Chrome in the Colab environment using wget and dpkg
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

# Set the path to the Chrome binary for webdriver_manager (though it often finds it automatically)
# This might not be strictly necessary, but good for debugging if issues persist
chrome_binary_path = "/usr/bin/google-chrome"
if os.path.exists(chrome_binary_path):
    os.environ['CHROME_PATH'] = chrome_binary_path
    print(f"Chrome binary found and CHROME_PATH set to: {os.environ['CHROME_PATH']}")
else:
    print(f"Chrome binary not found at expected path: {chrome_binary_path}")

print("Google Chrome installation process completed.")

**Reasoning**:
Google Chrome has been successfully installed, and the `CHROME_PATH` environment variable has been set. Now, I will re-run the test of the `download_table` function with a single URL to verify that Selenium can now properly find and use the Chrome binary to scrape the table data.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The previous attempt resulted in a generic Selenium error. I will modify the `download_table` function to explicitly pass the Chrome binary path to the `Service` constructor to ensure Selenium can find and use the installed Chrome browser, and add more specific error handling to diagnose potential `WebDriverException` issues.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")

        # Explicitly set the path to the Chrome binary
        chrome_service = Service(
            executable_path=ChromeDriverManager().install(), service_args=["--verbose"]
        )
        driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        # The selector here targets the table element itself, not its content
        wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The previous attempt resulted in a generic Selenium error. I will modify the `download_table` function to explicitly pass the Chrome binary path to the `Service` constructor to ensure Selenium can find and use the installed Chrome browser, and add more specific error handling to diagnose potential `WebDriverException` issues.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")

        # Explicitly set the path to the Chrome binary
        chrome_service = Service(
            executable_path=ChromeDriverManager().install(), service_args=["--verbose"]
        )
        driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        # The selector here targets the table element itself, not its content
        wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The `download_table` function has been updated with explicit Chrome binary path handling and enhanced error reporting. Now, I will re-run the test with a single URL to verify that Selenium can now properly find and use the Chrome binary to scrape the table data.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The previous attempt to run Selenium still resulted in an empty error message, indicating a potential issue with `ChromeDriverManager().install()` or how the service is being initialized. I will modify the `download_table` function to directly use the `chrome_binary_path` that was verified earlier and remove the `service_args=['--verbose']` for a cleaner log, hoping to resolve the WebDriver initialization issue.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")

        # Explicitly set the path to the Chrome binary and ChromeDriver
        # Use the global chrome_binary_path variable if it exists
        chromedriver_path = ChromeDriverManager().install()
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        # The selector here targets the table element itself, not its content
        wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The previous attempt to run Selenium still resulted in an empty error message, indicating a potential issue with `ChromeDriverManager().install()` or how the service is being initialized. I will modify the `download_table` function to directly use the `chrome_binary_path` that was verified earlier and remove the `service_args=['--verbose']` for a cleaner log, hoping to resolve the WebDriver initialization issue.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")

        # Explicitly set the path to the Chrome binary and ChromeDriver
        # Use the global chrome_binary_path variable if it exists
        chromedriver_path = ChromeDriverManager().install()
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        # The selector here targets the table element itself, not its content
        wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The previous attempts to initialize the Selenium WebDriver failed because the Chrome browser binary was not being found. To address this, I will explicitly set the `binary_location` for the Chrome options to the `chrome_binary_path` that was successfully installed and verified earlier, within the `download_table` function.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Explicitly set the path to the Chrome binary
        global chrome_binary_path  # Access the global variable
        if "CHROME_PATH" in os.environ:
            chrome_options.binary_location = os.environ["CHROME_PATH"]
        elif chrome_binary_path and os.path.exists(chrome_binary_path):
            chrome_options.binary_location = chrome_binary_path
        else:
            print(
                "Warning: Chrome binary path not explicitly set. Relying on default discovery."
            )

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")

        # Explicitly set the path to the Chrome driver
        chromedriver_path = ChromeDriverManager().install()
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        # The selector here targets the table element itself, not its content
        wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The `download_table` function has been updated to explicitly set the Chrome binary location. Now, I will re-run the test with a single URL to verify that Selenium can properly find and use the Chrome binary to scrape the table data.



In [ ]:
import pandas as pd
import time
import random

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The 'Read timed out' error suggests a communication issue between Selenium and the Chrome driver, possibly due to repeated driver installation within the loop or insufficient timeouts. I will modify the setup to initialize the ChromeDriver path once globally to avoid redundancy and explicitly set page load and script timeouts within the WebDriver options for better stability.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent

# Initialize ChromeDriverManager once globally
try:
    global chromedriver_path
    chromedriver_path = ChromeDriverManager().install()
    print(f"ChromeDriver installed/located at: {chromedriver_path}")
except Exception as e:
    print(f"Error installing ChromeDriver: {e}")
    chromedriver_path = None


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    if not chromedriver_path:
        print("ChromeDriver not available, cannot proceed with scraping.")
        return None

    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Explicitly set the path to the Chrome binary
        global chrome_binary_path  # Access the global variable
        if "CHROME_PATH" in os.environ:
            chrome_options.binary_location = os.environ["CHROME_PATH"]
        elif chrome_binary_path and os.path.exists(chrome_binary_path):
            chrome_options.binary_location = chrome_binary_path
        else:
            print(
                "Warning: Chrome binary path not explicitly set. Relying on default discovery."
            )

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")
        chrome_options.add_argument(
            "--remote-debugging-port=9222"
        )  # Recommended for Colab

        # Explicitly set the path to the Chrome driver and use longer timeouts
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.set_page_load_timeout(60)  # Set page load timeout to 60 seconds
        driver.set_script_timeout(60)  # Set script timeout to 60 seconds

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        wait = WebDriverWait(driver, 30)  # Increased WebDriverWait to 30 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The `download_table` function has been updated with global ChromeDriver initialization, explicit Chrome binary location, and increased timeouts. Now, I will re-run the test with a single URL to verify that Selenium can properly scrape the table data without timing out.



In [ ]:
import pandas as pd
import time
import random
import os  # Import os for environment variable checks

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

# Task
Okay, I will proceed with the plan.

First, I will modify the `download_table` function to enable verbose ChromeDriver logging. This will help diagnose the 'Read timed out' error previously encountered. I will add `service_args=['--verbose', '--log-path=/tmp/chromedriver.log']` to the `Service` initialization.

```python
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import os

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]' # Updated selector
ua = UserAgent()  # Initialize UserAgent

# Initialize ChromeDriverManager once globally
try:
    global chromedriver_path
    chromedriver_path = ChromeDriverManager().install()
    print(f"ChromeDriver installed/located at: {chromedriver_path}")
except Exception as e:
    print(f"Error installing ChromeDriver: {e}")
    chromedriver_path = None

def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None # Initialize driver to None for cleanup in finally block
    if not chromedriver_path:
        print("ChromeDriver not available, cannot proceed with scraping.")
        return None

    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')

        # Explicitly set the path to the Chrome binary
        global chrome_binary_path # Access the global variable
        if 'CHROME_PATH' in os.environ:
            chrome_options.binary_location = os.environ['CHROME_PATH']
        elif 'chrome_binary_path' in globals() and chrome_binary_path and os.path.exists(chrome_binary_path):
            chrome_options.binary_location = chrome_binary_path
        else:
            print("Warning: Chrome binary path not explicitly set. Relying on default discovery.")

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f'user-agent={new_user_agent}')
        chrome_options.add_argument('--remote-debugging-port=9222') # Recommended for Colab

        # Explicitly set the path to the Chrome driver and use longer timeouts
        # Enable verbose logging for ChromeDriver
        service = Service(executable_path=chromedriver_path, service_args=['--verbose', '--log-path=/tmp/chromedriver.log'])
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.set_page_load_timeout(60) # Set page load timeout to 60 seconds
        driver.set_script_timeout(60)    # Set script timeout to 60 seconds

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        wait = WebDriverWait(driver, 30) # Increased WebDriverWait to 30 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, 'html.parser')
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all('tr')
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all('th')]

        data = []
        for row in rows:
            cells = row.find_all('td')
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver: # Ensure driver is closed even if an error occurs
            driver.quit()

```

## Enable Verbose ChromeDriver Logging

### Subtask:
Modify the `download_table` function to enable verbose logging for the ChromeDriver service. This will provide detailed output on the WebDriver's interactions and help identify the cause of the 'Read timed out' error.


**Reasoning**:
To enable verbose logging for ChromeDriver and help diagnose the 'Read timed out' error, I will modify the `download_table` function in cell `d09e5139` by adding `service_args=['--verbose', '--log-path=/tmp/chromedriver.log']` to the `Service` initialization.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

# Import Selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

selector = 'table[width="100%"][cellpadding="3"][cellspacing="1"]'  # Updated selector
ua = UserAgent()  # Initialize UserAgent

# Initialize ChromeDriverManager once globally
try:
    global chromedriver_path
    chromedriver_path = ChromeDriverManager().install()
    print(f"ChromeDriver installed/located at: {chromedriver_path}")
except Exception as e:
    print(f"Error installing ChromeDriver: {e}")
    chromedriver_path = None


def download_table(url, selector):
    """
    Downloads table data from a Finviz page using Selenium with a headless Chrome browser.
    """
    driver = None  # Initialize driver to None for cleanup in finally block
    if not chromedriver_path:
        print("ChromeDriver not available, cannot proceed with scraping.")
        return None

    try:
        # Setup Chrome options for headless browsing
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Explicitly set the path to the Chrome binary
        global chrome_binary_path  # Access the global variable
        if "CHROME_PATH" in os.environ:
            chrome_options.binary_location = os.environ["CHROME_PATH"]
        elif chrome_binary_path and os.path.exists(chrome_binary_path):
            chrome_options.binary_location = chrome_binary_path
        else:
            print(
                "Warning: Chrome binary path not explicitly set. Relying on default discovery."
            )

        # Add a random User-Agent header to mimic a browser
        new_user_agent = ua.random
        chrome_options.add_argument(f"user-agent={new_user_agent}")
        chrome_options.add_argument(
            "--remote-debugging-port=9222"
        )  # Recommended for Colab

        # Explicitly set the path to the Chrome driver and use longer timeouts
        # Enable verbose logging for ChromeDriver
        service = Service(
            executable_path=chromedriver_path,
            service_args=["--verbose", "--log-path=/tmp/chromedriver.log"],
        )
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.set_page_load_timeout(60)  # Set page load timeout to 60 seconds
        driver.set_script_timeout(60)  # Set script timeout to 60 seconds

        print(f"Navigating to URL: {url}")
        driver.get(url)

        # Wait for the table element to be present (adjust timeout as needed)
        wait = WebDriverWait(driver, 30)  # Increased WebDriverWait to 30 seconds
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))

        # Retrieve the page source after JavaScript has rendered the content
        page_source = driver.page_source
        print(f"Page source retrieved. Length: {len(page_source)} bytes")

        soup = BeautifulSoup(page_source, "html.parser")
        table_body = soup.select_one(selector)

        if table_body is None:
            print(f"Error: Table body not found using selector: {selector}")
            return None

        rows = table_body.find_all("tr")
        if not rows:
            print("Error: No rows found in the table.")
            return None

        # Extract headers from the first row (th elements)
        headers_list = [th.text.strip() for th in rows[0].find_all("th")]

        data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = [cell.text.strip() for cell in cells]
            if row_data:  # Only append if the row has data
                data.append(row_data)

        if not data:
            print("Error: No data found in the table rows.")
            return None

        df = pd.DataFrame(data, columns=headers_list)
        return df

    except WebDriverException as e:
        print(f"An error occurred with the WebDriver: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Selenium scraping: {e}")
        return None
    finally:
        if driver:  # Ensure driver is closed even if an error occurs
            driver.quit()

**Reasoning**:
The `download_table` function has been updated with verbose ChromeDriver logging. Now, I will re-run the test with a single URL to verify if the logging provides insights into the 'Read timed out' error and if the table can be successfully scraped.



In [ ]:
import pandas as pd
import time
import random
import os  # Import os for environment variable checks

total_urls_to_download = len(shuffled_urls)

df = pd.DataFrame()  # Initialized an empty DataFrame
processed_count = 0

# Test the download_table function with a single URL to verify
url_to_test = shuffled_urls[0]
print(f"Testing download_table with URL: {url_to_test}")
df_test = download_table(url_to_test, selector)

if df_test is not None:
    print("Test download successful. Displaying first 5 rows of the test DataFrame:")
    print(df_test.head())
else:
    print("Test download failed. Please review the output above for errors.")

**Reasoning**:
The previous execution indicated a WebDriver error, but the verbose log file was not inspected. To diagnose the 'Read timed out' error and the generic WebDriver error, I need to read the `chromedriver.log` file that was generated by the verbose logging to get more specific details.



In [ ]:
log_file_path = "/tmp/chromedriver.log"
if os.path.exists(log_file_path):
    print(f"Content of {log_file_path}:\n")
    with open(log_file_path, "r") as f:
        print(f.read())
else:
    print(f"ChromeDriver log file not found at {log_file_path}.")